<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Project-Description" data-toc-modified-id="Project-Description-1">Project Description</a></span><ul class="toc-item"><li><span><a href="#Data-description" data-toc-modified-id="Data-description-1.1">Data description</a></span></li></ul></li><li><span><a href="#Import-Libraries" data-toc-modified-id="Import-Libraries-2">Import Libraries</a></span></li><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-3">Exploratory data analysis</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-3.1">Load data</a></span></li><li><span><a href="#Preprocessing-data" data-toc-modified-id="Preprocessing-data-3.2">Preprocessing data</a></span></li></ul></li><li><span><a href="#Split-Data" data-toc-modified-id="Split-Data-4">Split Data</a></span></li><li><span><a href="#Model-building" data-toc-modified-id="Model-building-5">Model building</a></span><ul class="toc-item"><li><span><a href="#geo_0-region" data-toc-modified-id="geo_0-region-5.1">geo_0 region</a></span></li><li><span><a href="#geo_1-region" data-toc-modified-id="geo_1-region-5.2">geo_1 region</a></span></li><li><span><a href="#geo_2-region" data-toc-modified-id="geo_2-region-5.3">geo_2 region</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5.4">Conclusion</a></span></li></ul></li><li><span><a href="#Profit-calculation" data-toc-modified-id="Profit-calculation-6">Profit calculation</a></span><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-6.1">Conclusion</a></span></li></ul></li><li><span><a href="#Profit-from-a-set-of-selected-oil-wells-and-model-predictions" data-toc-modified-id="Profit-from-a-set-of-selected-oil-wells-and-model-predictions-7">Profit from a set of selected oil wells and model predictions</a></span><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-7.1">Conclusion</a></span></li></ul></li><li><span><a href="#Calculate-risks-and-profit-for-each-region" data-toc-modified-id="Calculate-risks-and-profit-for-each-region-8">Calculate risks and profit for each region</a></span><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-8.1">Conclusion</a></span></li></ul></li><li><span><a href="#Overall-Conclusion" data-toc-modified-id="Overall-Conclusion-9">Overall Conclusion</a></span></li></ul></div>

# Project Description

---

Analyze potential profit and risks for an oil company.

Create a model that will analyze different proposed locations for a new oil well that will maximize profits while minimizing risk.

Goal is to find the best place for a new oil well using provided data from three different regions.

Steps to choose the location:
* Collect the oil well parameters in the selected region: oil quality and volume of reserves;
* Build a model for predicting the volume of reserves in the new wells;
* Pick the oil wells with the highest estimated values;
* Pick the region with the highest total profit for the selected oil wells.

There is data on oil samples from three regions. Parameters of each oil well in the region are already known. Build a model that will help to pick the region with the highest profit margin. Analyze potential profit and risks using the Bootstrapping technique.

## Data description

---

Geological exploration data for the three regions are stored in files:
* *id* — unique oil well identifier
* *f0, f1, f2* — three features of points (their specific meaning is unimportant, but the features themselves are significant)
* *product* — volume of reserves in the oil well (thousand barrels).

**Conditions:**
* Only linear regression is suitable for model training (the rest are not sufficiently predictable).
* When exploring the region, a study of 500 points is carried with picking the best 200 points for the profit calculation.
* The budget for development of 200 oil wells is 100 USD million.
* One barrel of raw materials brings 4.5 USD of revenue The revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels).
* After the risk evaluation, keep only the regions with the risk of losses lower than 2.5%. From the ones that fit the criteria, the region with the highest average profit should be selected.

The data is synthetic: contract details and well characteristics are not disclosed.

# Import Libraries

In [ ]:
# Import in libraries to use in project

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from tabulate import tabulate
from scipy import stats as st

# Exploratory data analysis

## Load data

In [ ]:
geo_0 = pd.read_csv('/content/datasets/geo_data_0.csv')
geo_1 = pd.read_csv('/content/datasets/geo_data_1.csv')
geo_2 = pd.read_csv('/content/datasets/geo_data_2.csv')

In [ ]:
# Functions to get descriptions and info from dataframe

def get_information(df):
  """ Prints general info about the dataframe to get an idea of what it looks like"""
  print('Head: \n')
  display(df.head())
  print('*'*100, '\n') # Prints a break to seperate print data
  
  print('Info: \n')
  display(df.info())
  print('*'*100, '\n')

  print('Describe: \n')
  display(df.describe())
  print('*'*100, '\n')

  print('Columns with nulls: \n')
  display(get_null_df(df,4))
  print('*'*100, '\n')

  print('Shape: \n')
  display(df.shape)
  print('*'*100, '\n')

  print('Duplicated: \n')
  print('Number of duplicated rows: {}'.format(df.duplicated().sum()))

def get_null_df(df, num):
  """Gets percentage of null values per column per dataframe"""
  df_nulls = pd.DataFrame(df.isna().sum(), columns=['missing_values'])
  df_nulls['percent_of_nulls'] = round(df_nulls['missing_values'] / df.shape[0], num) *100
  return df_nulls

def get_null(df):
  """Gets percentage of null values in dataframe"""
  count = 0
  df = df.copy()
  s = (df.isna().sum() / df.shape[0])
  for column, percent in zip(s.index, s.values):

    num_of_nulls = df[column].isna().sum()
    if num_of_nulls == 0:
      continue
    else:
      count += 1
    print('Columns {} has {:.{}%} percent of Nulls, and {} number of nulls'.format(column, percent, num, num_of_nulls))

    if count !=0:
      print('Number of columns with NA: {}'.format(count))
    else:
      print('\nNo NA columns found')

In [ ]:
#Opening dataset for geo_0
get_information(geo_0)

Head: 



,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


**************************************************************************************************** 

Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

**************************************************************************************************** 

Describe: 



...

**************************************************************************************************** 

Columns with nulls: 



,missing_values,percent_of_nulls
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


**************************************************************************************************** 

Shape: 



(100000, 5)

**************************************************************************************************** 

Duplicated: 

Number of duplicated rows: 0


Dataset for `geo_0` show to have 100000 entries with no null/missing values and datatypes seem to be correct.

In [ ]:
#Opening dataset for geo_1
get_information(geo_1)

Head: 



,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


**************************************************************************************************** 

Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

**************************************************************************************************** 

Describe: 



...

**************************************************************************************************** 

Columns with nulls: 



,missing_values,percent_of_nulls
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


**************************************************************************************************** 

Shape: 



(100000, 5)

**************************************************************************************************** 

Duplicated: 

Number of duplicated rows: 0


Dataset for `geo_1` show to have 100000 entries with no null/missing values and datatypes seem to be correct.

In [ ]:
#Opening dataset for geo_2
get_information(geo_2)

Head: 



,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**************************************************************************************************** 

Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

**************************************************************************************************** 

Describe: 



...

**************************************************************************************************** 

Columns with nulls: 



,missing_values,percent_of_nulls
id,0,0.0
f0,0,0.0
f1,0,0.0
f2,0,0.0
product,0,0.0


**************************************************************************************************** 

Shape: 



(100000, 5)

**************************************************************************************************** 

Duplicated: 

Number of duplicated rows: 0


Dataset for `geo_2` show to have 100000 entries with no null/missing values and datatypes seem to be correct.

From all the datasets, we might be able to drop the **id** column as it does not provide the models much information.

## Preprocessing data

In [ ]:
#Remove columns that do not provide unique or relational information that our model could use
geo_0_data = geo_0.drop(['id'], axis=1)
display(geo_0_data.head())

geo_1_data = geo_1.drop(['id'], axis=1)
display(geo_1_data.head())

geo_2_data = geo_2.drop(['id'], axis=1)
display(geo_2_data.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


The datasets `geo_0`, `geo_1`, `geo_2` all have 100,000 entries with no missing values and have datatypes that seem to be correct. We were able to drop the **id** column as this would not provide much information for our model.

# Split Data

In [ ]:
RANDOM_STATE = 12345 #Random_State

def get_train_valid(df):
    df_train, df_valid = train_test_split(df, test_size=0.25, random_state=RANDOM_STATE) # Splits data up to 75% train and 25% test
    return df_train, df_valid

In [ ]:
geo0_target = geo_0_data['product']
geo0_features = geo_0_data.drop(['product'], axis=1)

geo0_x_train, geo0_x_valid = get_train_valid(geo0_features)
geo0_y_train, geo0_y_valid = get_train_valid(geo0_target)

assert geo0_x_train.shape[0] == geo0_y_train.shape[0]
assert geo0_x_valid.shape[0] == geo0_y_valid.shape[0]

print('geo_0 datasets: \n')
print('Train:', geo0_x_train.shape, ' Target Train:', geo0_y_train.shape)
print('Validation:', geo0_x_valid.shape, ' Target Validation:', geo0_y_valid.shape)

geo_0 datasets: 

Train: (75000, 3)  Target Train: (75000,)
Validation: (25000, 3)  Target Validation: (25000,)


In [ ]:
geo1_target = geo_1_data['product']
geo1_features = geo_1_data.drop(['product'], axis=1)

geo1_x_train, geo1_x_valid = get_train_valid(geo1_features)
geo1_y_train, geo1_y_valid = get_train_valid(geo1_target)

assert geo1_x_train.shape[0] == geo1_y_train.shape[0]
assert geo1_x_valid.shape[0] == geo1_y_valid.shape[0]

print('geo_1 datasets: \n')
print('Train:', geo1_x_train.shape, ' Target Train:', geo1_y_train.shape)
print('Validation:', geo1_x_valid.shape, ' Target Validation:', geo1_y_valid.shape)

geo_1 datasets: 

Train: (75000, 3)  Target Train: (75000,)
Validation: (25000, 3)  Target Validation: (25000,)


In [ ]:
geo2_target = geo_2_data['product']
geo2_features = geo_2_data.drop(['product'], axis=1)

geo2_x_train, geo2_x_valid = get_train_valid(geo2_features)
geo2_y_train, geo2_y_valid = get_train_valid(geo2_target)

assert geo2_x_train.shape[0] == geo2_y_train.shape[0]
assert geo2_x_valid.shape[0] == geo2_y_valid.shape[0]

print('geo_2 datasets: \n')
print('Train:', geo2_x_train.shape, ' Target Train:', geo2_y_train.shape)
print('Validation:', geo2_x_valid.shape, ' Target Validation:', geo2_y_valid.shape)

geo_2 datasets: 

Train: (75000, 3)  Target Train: (75000,)
Validation: (25000, 3)  Target Validation: (25000,)


<div class="alert alert-block alert-danger">
<b>Needs fixing:</b> We don't need valid part in the task because we don't tune parameters. Create only train(75%) and test(25%).
</div>

***Updated to only have validation(25%) and train(75%) datasets, since we do not need the test dataset for tunning parameters on our models***

The dataset for the three different regions were successfully split into train and validation datasets with a 75:25 ratio. There is no testing sub-dataset as the models will not be hyper-tuned.

# Model building

In [ ]:
def LinReg_sanity_check(x_train, x_valid, y_train, y_valid):
    model = LinearRegression()
    model.fit(x_train, y_train)
    
    predictions = model.predict(x_valid)

    print('---Model validation/prediction datasets scores---')
    print('Accuracy:', model.score(x_valid, y_valid))
    print('R2:', r2_score(y_valid, predictions))
    print('RMSE:', mean_squared_error(y_valid, predictions, squared=False))


## geo_0 region

In [ ]:
geo0_model = LinearRegression()
geo0_model.fit(geo0_x_train, geo0_y_train)
geo0_predictions = geo0_model.predict(geo0_x_valid)

print('Sanity check for geo_0 model:')
LinReg_sanity_check(geo0_x_train, geo0_x_valid, geo0_y_train, geo0_y_valid)
print('\nAverage volume of predicted reserves in geo_0 (thousand barrels):', geo0_predictions.mean())

Sanity check for geo_0 model:
---Model validation/prediction datasets scores---
Accuracy: 0.27994321524487786
R2: 0.27994321524487786
RMSE: 37.5794217150813

Average volume of predicted reserves in geo_0 (thousand barrels): 92.59256778438038


For the data in geo_0, the accuracy, R2 and RMSE scores are pretty terrble. The accuracy and R2 score are very low and the RMSE score is very big. 

## geo_1 region

In [ ]:
geo1_model = LinearRegression()
geo1_model.fit(geo1_x_train, geo1_y_train)
geo1_predictions = geo1_model.predict(geo1_x_valid)

print('Sanity check for geo_1 model:\n')
LinReg_sanity_check(geo1_x_train, geo1_x_valid, geo1_y_train, geo1_y_valid)
print('\nAverage volume of predicted reserves in geo_1 (thousand barrels):', geo1_predictions.mean())

Sanity check for geo_1 model:

---Model validation/prediction datasets scores---
Accuracy: 0.9996233978805126
R2: 0.9996233978805127
RMSE: 0.893099286775616

Average volume of predicted reserves in geo_1 (thousand barrels): 68.728546895446


The scores for data in geo_1 are actually pretty good with accuracy and R2 score pretty close to 1 and RMSE score being very low. 

## geo_2 region

In [ ]:
geo2_model = LinearRegression()
geo2_model.fit(geo2_x_train, geo2_y_train)
geo2_predictions = geo2_model.predict(geo2_x_valid)

print('Sanity check for geo_2 model:\n')
LinReg_sanity_check(geo2_x_train, geo2_x_valid, geo2_y_train, geo2_y_valid)
print('\nAverage volume of predicted reserves in geo_2 (thousand barrels):', geo2_predictions.mean())

Sanity check for geo_2 model:

---Model validation/prediction datasets scores---
Accuracy: 0.20524758386040443
R2: 0.20524758386040443
RMSE: 40.02970873393434

Average volume of predicted reserves in geo_2 (thousand barrels): 94.96504596800489


Similar to the model for geo_0, the scores here for geo_2 model are not that great with a very low accuracy and R2 score and high RMSE score. 

## Conclusion

The table below shows each models' results using the valdiation datasets to make predictions:

---

|  | geo_0 | geo_1 | geo_2 |
| :-----------|:-----------|:-----------|:-----------|
| Accuracy Score | 0.2799 | 0.9996 | 0.2052 |
| R2 Score | 0.2799 | 0.9996 | 0.2052 |
| RMSE Score | 37.579 | 0.8903 | 40.030 |
| Average **predicted** volume reserves (thousand barrels)| 92.592 | 68.728 | 94.965 |

---

R2 score shows the relative measure of fit, while RMSE is an absolute measure of fit. Lower values of RMSE indicate a better fit. 

The model for geo_1 has the best scores for accuracy, R2, and RMSE. With accuracy and R2 being close to 1 and with a very small RMSE it seems like the model can predict pretty well using the dataset provided for geo_1. However, predicted average volume of reserves in ge0_1 is the lowest of the 3 regions being at 68.968 thousands barrels.

The models for geo_0 and geo_2 performed pretty similar for the datasets given for these two regions. The accuracy and R2 score are very small ranging around 0.20 - 0.30 and have RMSE score around 40. In comparison, though these two have similar scores in accuracy, R2 and RMSE, the model for geo_0 did slightly better with a slightly higher accuracy and R2 score and a bit lower RMSE score compared to the model for geo_2. 

The model for geo_2 predicted largest average volume of reserves with 94.955 thousands barrels. With geo_0 coming in second with the largest predicted average volume of reserves at 92.708 thousands barrels. geo_1 came in last with the smallest predicted average volume at 68.968 thousand barrels.

Having such a large RMSE score for the models for geo_0 and geo_2 shows that perhaps a linear regression model might not be ideal for these two regions.

# Profit calculation

In [ ]:
BUDGET = 100000000 # budget for development of 200 oil wells is 100 USD million
POINT_PER_BUDGET = 200 # number of wells in the budget
PRODUCT_PRICE = 4500 # revenue from one unit of product is 4,500 dollars (volume of reserves is in thousand barrels)

#Calculation for the volume of reserves sufficient for developing a new well without losses
volume_no_loss = (BUDGET/POINT_PER_BUDGET) / PRODUCT_PRICE

print('Volume of reserves sufficient for developing a new well without losses:', volume_no_loss, '(thousand barrels)')

def average_volume(df):
  return df['product'].mean()

print('\nAverage volume of reserves in each region:')
print('* geo_0: ', average_volume(geo_0), '(thousand barrels)')
print('* geo_1: ', average_volume(geo_1), '(thousand barrels)')
print('* geo_2: ', average_volume(geo_2), '(thousand barrels)')

Volume of reserves sufficient for developing a new well without losses: 111.11111111111111 (thousand barrels)

Average volume of reserves in each region:
* geo_0:  92.49999999999976 (thousand barrels)
* geo_1:  68.82500000002561 (thousand barrels)
* geo_2:  95.00000000000041 (thousand barrels)


## Conclusion

The minimum volume of reserves need to develop a new well without losses is 111.11 (thousand barrels). 

Tables showing average volume in each region:

|  | geo_0 | geo_1 | geo_2 |
| :-----------|:-----------|:-----------|:-----------|
| Average volume of reserves (thousand barrels)| 92.499 | 68.825 | 95.00 |
| Average **predicted** volume reserves (thousand barrels)| 92.592 | 68.728 | 94.965 |

Just by looking at the average volume of reserves in reach region (actual and predicted volumes), it doesn't seem that a developing a new well would be profitable. We would have to look at the top producing wells (actual and predicted) in each region to see if the calculation differs.


# Profit from a set of selected oil wells and model predictions

In [ ]:
def revenue(y_valid, predictions, count):

    predictions = pd.Series(predictions)
    y_valid = pd.Series(y_valid.values)

    predict_sorted = predictions.sort_values(ascending=False)
    selected_wells = y_valid[predict_sorted.index][:count]
    return PRODUCT_PRICE * selected_wells.sum()

def profit(revenue):
  
  profit = revenue - BUDGET
  return profit

In [ ]:
# Using top 200 wells from region geo_0 with predicted volumes/actual volumes to calculate profit
rev_geo0 = revenue(geo0_y_valid, geo0_predictions, 200)
print('Predicted profit for region geo_0 with top best 200 wells:', profit(rev_geo0).round(2))

Predicted profit for region geo_0 with top best 200 wells: 33208260.43


In [ ]:
# Using top 200 wells from region geo_1 with predicted volumes/actual volumes to calculate profit
rev_geo1 = revenue(geo1_y_valid, geo1_predictions, 200)
print('Predicted profit for region geo_1 with top best 200 wells:', profit(rev_geo1).round(2))

Predicted profit for region geo_1 with top best 200 wells: 24150866.97


In [ ]:
# Using top 200 wells from region geo_2 with predicted volumes/actual volumes to calculate profit
rev_geo2 = revenue(geo2_y_valid, geo2_predictions, 200)
print('Predicted profit for region geo_2 with top best 200 wells:', profit(rev_geo2).round(2))

Predicted profit for region geo_2 with top best 200 wells: 27103499.64


## Conclusion

From the calculations in this section, it's best to develop new wells in region geo_0. The calculation for predicted profit is highest in this region at 33,208,260.43 USD. geo_1 had the lowest predicted profit at 24,150,866.97 USD and geo_2 came out in the middle at 27,103,499.64 USD.

# Calculate risks and profit for each region

In [ ]:
#Function to get values for profit distribution using bootstrapping with 1000 samples
def profit_distribution(y_valid, predictions):

    y_valid = pd.Series(y_valid.values)

    state = np.random.RandomState(12345)
    
    values = []
    for i in range(1000):

        target_subsample = y_valid.sample(n=500, replace=True, random_state=state)
        predict_subsample = predictions[target_subsample.index] 
        rev = revenue(target_subsample, predict_subsample, 200)
        values.append(rev)

    values = pd.Series(values)
    values = values.sort_values()

    profit_values = profit(values)

    return profit_values

#Calculation for confidence interval at 95%
def confidence_interval(profit_values):
  confidence_interval = st.t.interval(
      0.95, len(profit_values)-1, profit_values.mean(), profit_values.sem()) 
  
  return print("95% confidence interval:", confidence_interval)


#Value at Risk (Risk of Losses) | Confidence Level calculation
def risk_of_loss(profit_values):
  """
  Value at risk (VaR) is a measure of the risk of loss for investments. 
  It estimates how much a set of investments might lose (with a given probability), 
  given normal market conditions, in a set time period such as a day
  """
  upper = profit_values.quantile(0.975)
  lower = profit_values.quantile(0.025)

  return print('\n', tabulate([['2.5%', lower], ['97.5%', upper]], floatfmt='.2f',
                        headers=['Confidence Level', 'Value at Risk']))

#Sum of loss (negative profit) within our 1000 sample
def loss(profit_values):
  loss = sum(i for i in profit_values if i < 0)
  return loss

#Get number of instances where profit sample is negative
def count_loss(profit_values):
  num_loss = profit_values.lt(0).sum().sum()
  return num_loss

#Sum of gain (positive profit) within our 1000 sample
def gain(profit_values):
  gain = sum(i for i in profit_values if i > 0)
  return gain

#Get number of instances where profit sample is positive
def count_gain(profit_values):
  num_gain = profit_values.gt(0).sum().sum()
  return num_gain


#Calculation for profit/loss ratio
def proft_loss_ratio(profit_values):
  total_gain = gain(profit_values)
  total_loss = loss(profit_values)

  total_num_gain = count_gain(profit_values)
  total_num_loss = count_loss(profit_values)

  ratio = ((total_gain/total_num_gain) / (total_loss/total_num_loss))

  return ratio

#Loss probability calculation
def loss_prob(profit_values):
  """
  Loss probability is percentage of negative values in profit array
  """
  total_num_loss = count_loss(profit_values) #Count of negative values in profit array
  prob = total_num_loss / 1000 #Sample size of 1000 for profit array

  return prob

In [ ]:
#Calculations for geo_0 region:
geo0_profit_values = profit_distribution(geo0_y_valid, geo0_predictions)

print('Average profit for region geo_0:', round(geo0_profit_values.mean(), 2))

confidence_interval(geo0_profit_values)

risk_of_loss(geo0_profit_values)

#Loss probability
print('\nLoss probability for geo_0:', loss_prob(geo0_profit_values), 'or', 
      '{:.2%}'.format(loss_prob(geo0_profit_values)))


Average profit for region geo_0: 3961649.85
95% confidence interval: (3796203.151479729, 4127096.544567701)

 Confidence Level      Value at Risk
------------------  ---------------
2.5%                    -1112155.46
97.5%                    9097669.42

Loss probability for geo_0: 0.069 or 6.90%


In [ ]:
#Calculations for geo_1 region:
geo1_profit_values = profit_distribution(geo1_y_valid, geo1_predictions)

print('Average profit for region geo_1:', round(geo1_profit_values.mean(), 2))

confidence_interval(geo1_profit_values)

risk_of_loss(geo1_profit_values)

#Loss probability
print('\nLoss probability for geo_1:', loss_prob(geo1_profit_values), 'or', 
      '{:.2%}'.format(loss_prob(geo1_profit_values)))


Average profit for region geo_1: 4560451.06
95% confidence interval: (4431472.486639012, 4689429.629094217)

 Confidence Level      Value at Risk
------------------  ---------------
2.5%                      338205.09
97.5%                    8522894.54

Loss probability for geo_1: 0.015 or 1.50%


In [ ]:
#Calculations for geo_2 region:
geo2_profit_values = profit_distribution(geo2_y_valid, geo2_predictions)

print('Average profit for region geo_2:', round(geo2_profit_values.mean(), 2))

confidence_interval(geo2_profit_values)

risk_of_loss(geo2_profit_values)

#Loss probability
print('\nLoss probability for geo_2:', loss_prob(geo2_profit_values), 'or', 
      '{:.2%}'.format(loss_prob(geo2_profit_values)))

Average profit for region geo_2: 4044038.67
95% confidence interval: (3874457.974712804, 4213619.356654332)

 Confidence Level      Value at Risk
------------------  ---------------
2.5%                    -1633504.13
97.5%                    9503595.75

Loss probability for geo_2: 0.076 or 7.60%


## Conclusion

Based on the calculations in this section, the best region get develop new oil wells would be region geo_1. Out of the 1000 samples that was obtained in each region, geo_1 had the smallest probablity of loss at 1.50%. The calculated average profit for region geo_1 out of the 1000 sample slice came in at 4,560,451.06 USD which was the highest average profit out of the three regions. The table below summarizes the findings in this secion:

|  | geo_0 | geo_1 | geo_2 |
| :-----------|:-----------|:-----------|:-----------|
| Average profit (USD) | 3,961,649.85 | 4,560,451.06 | 4,044,038.67 |
| 95% confidence interval | (3796203.15, 4127096.54) | (4431472.49, 4689429.63) | (3874457.97, 4213619.36) |
| Risk of loss at 2.5% confidence level | -1112155.46 | 338205.09 | -1633504.13 |
| Risk of loss at 97.5% confidence level | 9097669.42 | 8522894.54 | 9503595.75 |
| Loss probability (%) | 6.90% | 1.50% | 7.60% | 

# Overall Conclusion

In conlusion, with the calculations made for the provided datasets it seems as though region geo_1 is the best region develop new wells. The model for geo_1 had the best scores. The predicted profit for geo_1 came in second out of the three regions which isn't bad. And overall, geo_1 when sampled 1000 wells had the lowest loss probablity and had highest average profit. 

The table below summarize the sections we calculated:

|  | geo_0 | geo_1 | geo_2 |
| :-----------|:-----------|:-----------|:-----------|
| Accuracy Score | 0.2799 | 0.9996 | 0.2052 |
| R2 Score | 0.2799 | 0.9996 | 0.2052 |
| RMSE Score | 37.579 | 0.8903 | 40.030 |
| Average volume of reserves (thousand barrels)| 92.499 | 68.825 | 95.00 |
| Average **predicted** volume reserves (thousand barrels)| 92.592 | 68.728 | 94.965 |
| Predicted profit (USD) | 33,208,260.43 | 24,150,866.97 | 27,103,499.64 |
| Average profit (USD) | 3,961,649.85 | 4,560,451.06 | 4,044,038.67 |
| 95% confidence interval | (3796203.15, 4127096.54) | (4431472.49, 4689429.63) | (3874457.97, 4213619.36) |
| Risk of loss at 2.5% confidence level | -1112155.46 | 338205.09 | -1633504.13 |
| Risk of loss at 97.5% confidence level | 9097669.42 | 8522894.54 | 9503595.75 |
| Loss probability (%) | 6.90% | 1.50% | 7.60% |